In [1]:
import numpy as np
import torch
from microgrid_RL_agents import DQN_Agent
from microgrid_env import MicroGrid
from microgrid_fun import build_stacked_input, build_stacked_input_v2, build_stacked_input_zeropad
from microgrid_fun import hybrid_fhocp, qp_feasible, gurobi_qp
from utils import get_optimality_gap

import time

In [2]:
#action dict for SL

action_dict_SL = {'0': np.array([[0,0,0,0,0]]),
               '1': np.array([[0,0,0,0,1]]),
               '2': np.array([[0,0,0,1,0]]),
               '3': np.array([[0,0,0,1,1]]),
               '4': np.array([[0,0,1,0,0]]),
               '5': np.array([[0,0,1,0,1]]),
               '6': np.array([[0,0,1,1,0]]),
               '7': np.array([[0,0,1,1,1]]),
               '8': np.array([[0,1,0,0,0]]),
               '9': np.array([[0,1,0,0,1]]),
               '10': np.array([[0,1,0,1,0]]),
               '11': np.array([[0,1,0,1,1]]),
               '12': np.array([[0,1,1,0,0]]),
               '13': np.array([[0,1,1,0,1]]),
               '14': np.array([[0,1,1,1,0]]),
               '15': np.array([[0,1,1,1,1]]),
               '16': np.array([[1,0,0,0,0]]),
               '17': np.array([[1,0,0,0,1]]),
               '18': np.array([[1,0,0,1,0]]),
               '19': np.array([[1,0,0,1,1]]),
               '20': np.array([[1,0,1,0,0]]),
               '21': np.array([[1,0,1,0,1]]),
               '22': np.array([[1,0,1,1,0]]),
               '23': np.array([[1,0,1,1,1]]),
               '24': np.array([[1,1,0,0,0]]),
               '25': np.array([[1,1,0,0,1]]),
               '26': np.array([[1,1,0,1,0]]),
               '27': np.array([[1,1,0,1,1]]),
               '28': np.array([[1,1,1,0,0]]),
               '29': np.array([[1,1,1,0,1]]),
               '30': np.array([[1,1,1,1,0]]),
               '31': np.array([[1,1,1,1,1]])}

action_dict_RL = {'0': np.array([[0,0,0,0,0]]),
               '1': np.array([[1,0,0,0,0]]),
               '2': np.array([[0,1,0,0,0]]),
               '15': np.array([[1,1,0,0,0]]),
               '3': np.array([[0,0,1,0,0]]),
               '4': np.array([[0,0,1,1,0]]),
               '5': np.array([[0,0,1,1,1]]),
               '6': np.array([[1,0,1,0,0]]),
               '7': np.array([[1,0,1,1,0]]),
               '8': np.array([[1,0,1,1,1]]),
               '9': np.array([[0,1,1,0,0]]),
               '10': np.array([[0,1,1,1,0]]),
               '11': np.array([[0,1,1,1,1]]),
               '12': np.array([[1,1,1,0,0]]),
               '13': np.array([[1,1,1,1,0]]),
               '14': np.array([[1,1,1,1,1]])}

def build_delta_vector(list_action, N, action_dict):
    
    '''
    builds delta vector (N,5) from LSTM Network output
    '''
    
    # from list of actions builds a np.array with the stacked deltas for each time step of the prediction horizon
        
    delta = action_dict[str(list_action[0])]
    for i in range(1,N):
        delta = np.concatenate((delta, action_dict[str(list_action[i])]))
        
    return delta

In [9]:
import time
from microgrid_env import MicroGrid
from microgrid_fun import Network, state_norm

# N=48
N_sim=3000

data = np.load('data_costs_loads_2021_2022.npy', allow_pickle=True)
data_2021 = data[0]; data_2022 = data[1]
cbuy, csell, cprod, power_load, power_res = data_2022
cbuy_2021, csell_2021, cprod_2021, power_load_2021, power_res_2021 = data_2021

# for N in [12,24,48]:
for N in [12,24]:

    seed=42
    torch.manual_seed(seed)
    np.random.seed(seed)

    Env = MicroGrid(N)

    #parameter for RL
    input_size= 6 # for opt1
    hidden_size= 128
    num_layers = 1
    lr = 1e-4
    n_actions = 32
    batch_size = 1

    h0_SL = torch.zeros(num_layers, batch_size, hidden_size)
    c0_SL = torch.zeros(num_layers, batch_size, hidden_size)

    network_SL = Network(input_size, hidden_size, num_layers, lr, n_actions, batch_first=True)
    network_SL.load_state_dict(torch.load('best_weights//microgrid_lstm_supervised_%d_N%d_opt1' %(hidden_size,N)))
    #

    #parameters for RL
    num_layers = 1
    lr = 1e-4
    n_actions = 16
    batch_size = 1

    match N:
        case 4:
            hidden_size = 64
            state_opt = 5
        case 12:
            hidden_size = 256
            state_opt = 4
        case 24:
            hidden_size = 512
            state_opt = 2
        case 48:
            hidden_size = 512
            state_opt = 4

    match state_opt:
        case 1|2:
            input_size=6
        case 3|4:
            input_size=4*N+1
        case 5:
            input_size=5*N+1

    h0_RL = torch.zeros(num_layers, 1, hidden_size)
    c0_RL = torch.zeros(num_layers, 1, hidden_size)

    network_RL = Network(input_size, hidden_size, num_layers, lr, n_actions, batch_first=True)

    match N:
        case 4:
            network_RL.load_state_dict(torch.load('best_weights\job_N04_029_24_2_24_16_47_22'))
        case 12:
            # network_RL.load_state_dict(torch.load('best_weights\job_N12_27'))
            network_RL.load_state_dict(torch.load('best_weights\job_N12_054_24_2_27_15_6_47'))
        case 24:
            # network_RL.load_state_dict(torch.load('best_weights\job_N24_19_23_12_6_17_44_19'))
            network_RL.load_state_dict(torch.load('best_weights\job_N24_067_24_2_27_13_45_27'))
        case 48:       
            network_RL.load_state_dict(torch.load('best_weights\job_N48_103_24_2_6_2_26_14'))
    #

    list_SL = []
    list_RL = []
    list_milp = []

    # optgap_SL_1 = []
    # optgap_RL_1 = []
    # optgap_SL_2 = []
    # optgap_RL_2 = []
    
    list_cost_RL1 = []
    list_cost_RL2 = []
    list_cost_SL1 = []
    list_cost_SL2 = []
    list_cost_milp1 = []
    list_cost_milp2 = []
    list_cost_milp3 = []

    cntr_infeas_SL = 0
    cntr_infeas_RL = 0

    state_min_SL = np.array([25, np.min(cbuy), np.min(cprod), np.min(csell), np.min(power_load), np.min(power_res)])
    state_max_SL = np.array([250, np.max(cbuy), np.max(cprod), np.max(csell), np.max(power_load), np.max(power_res)])

    match state_opt:
        case 2:
            state_min = np.array([25, 0, 0, 0, np.min([np.min(power_load),np.min(power_res)]), np.min([np.min(power_load),np.min(power_res)])])
            state_max = np.array([250, 1, 1, 1, np.max([np.max(power_load),np.max(power_res)]), np.max([np.max(power_load),np.max(power_res)])])
        case 3:
            state_min = np.concatenate([
                np.array([25]), np.tile(np.min(cbuy),N), np.tile(np.min(csell),N), np.tile(np.min(cprod),N),
                np.tile(np.array([np.min(power_load-power_res)]),N)
            ])
            state_max = np.concatenate([
                np.array([250]), np.tile(np.max(cbuy),N), np.tile(np.max(csell),N), np.tile(np.max(cprod),N),
                np.tile(np.array([np.max(power_load-power_res)]),N)
            ])
        case 4:
            state_min = np.concatenate([
                np.array([25]), np.tile(np.array([0]),N), np.tile(np.array([0]),N), np.tile(np.array([0]),N),
                np.tile(np.array([np.min(power_load-power_res)]),N)
            ])
            state_max = np.concatenate([
                np.array([250]), np.tile(np.array([1]),N), np.tile(np.array([1]),N), np.tile(np.array([1]),N),
                np.tile(np.array([np.max(power_load-power_res)]),N)
            ])
        case 5:
            state_min = np.concatenate([
                np.array([25]), np.tile(np.array([np.min(cbuy)]),N), np.tile(np.array([np.min(csell)]),N), np.tile(np.array([np.min(cprod)]),N),
                np.tile(np.array([np.min(power_load)]),N), np.tile(np.array([np.min(power_res)]),N)
            ])
            state_max = np.concatenate([
                np.array([250]), np.tile(np.array([np.max(cbuy)]),N), np.tile(np.array([np.max(csell)]),N), np.tile(np.array([np.max(cprod)]),N),
                np.tile(np.array([np.max(power_load)]),N), np.tile(np.array([np.max(power_res)]),N)
            ]) 

    for i in range(N_sim):
        
        Env.set_randState()
        x0 = Env.state
        i = Env.idx_cntr
        
        x0_tmp = np.repeat(x0, N)
        # cbuy_tmp = cbuy_2021[i:i+N]
        # csell_tmp = csell_2021[i:i+N]
        # cprod_tmp = cprod_2021[i:i+N]
        # power_res_tmp = power_res_2021[i:i+N]
        # power_load_tmp = power_load_2021[i:i+N]
        
        cbuy_tmp = cbuy[i:i+N]
        csell_tmp = csell[i:i+N]
        cprod_tmp = cprod[i:i+N]
        power_res_tmp = power_res[i:i+N]
        power_load_tmp = power_load[i:i+N]

        state_SL = np.dstack((x0_tmp, cbuy_tmp, csell_tmp, cprod_tmp, power_res_tmp, power_load_tmp)).squeeze()
        state_SL = state_norm(state_SL, state_min_SL, state_max_SL).astype(np.float32)
        
        start_time_SL = time.time()
        state_SL = torch.tensor(state_SL, dtype=torch.float32).unsqueeze(0)
        output_net = network_SL(state_SL, h0_SL, c0_SL)
        action_idx = torch.max(output_net, dim=2)[1].squeeze().numpy()
        delta_SL = build_delta_vector(action_idx, N, action_dict_SL)
        end_time_SL = time.time()
            
        mdl_SL = gurobi_qp(x0, N, power_res_tmp, power_load_tmp, cbuy_tmp, csell_tmp, cprod_tmp, delta_SL)    
        
        match state_opt:
            case 1|2:
                state_RL = build_stacked_input(x0, i, N, input_size, state_min=state_min, state_max=state_max,
                                    cbuy=cbuy, csell=csell, cprod=cprod, power_load=power_load, power_res=power_res)
            case 3|4:
                state_RL = build_stacked_input_v2(x0, i, N, state_min2=state_min, state_max2=state_max,
                                            cbuy=cbuy, csell=csell, cprod=cprod, power_load=power_load, power_res=power_res)
            case 5:
                state_RL = build_stacked_input_zeropad(x0, i, N, state_min2=state_min, state_max2=state_max,
                                            cbuy=cbuy, csell=csell, cprod=cprod, power_load=power_load, power_res=power_res)
        
        start_time_RL = time.time()
        state_RL = torch.tensor(state_RL, dtype=torch.float32)
        output_net = network_RL(state_RL, h0_RL, c0_RL)
        action_idx = torch.max(output_net, dim=2)[1].squeeze(0).numpy()                        
        delta_RL = build_delta_vector(action_idx, N, action_dict_RL)
        end_time_RL = time.time()
        
        mdl_RL = gurobi_qp(x0, N, power_res_tmp, power_load_tmp, cbuy_tmp, csell_tmp, cprod_tmp, delta_RL)    
        
        mdl_milp = hybrid_fhocp(x0, N, power_res_tmp, power_load_tmp, cbuy_tmp, csell_tmp, cprod_tmp)
        
        feas_SL = qp_feasible(mdl_SL); feas_RL = qp_feasible(mdl_RL)   
        
        cost_milp = mdl_milp.ObjVal
        
        if feas_SL == True:
            cost_SL = mdl_SL.ObjVal - np.max(csell_tmp)*(mdl_SL.getVars()[N].x - mdl_milp.getVars()[N].x)
            # optgap_SL_1.append((np.abs(cost_SL-cost_milp))*100/np.abs(cost_milp))
            
            list_cost_SL1.append(cost_SL)
            list_cost_milp1.append(cost_milp)
            
        if feas_RL == True:
            cost_RL = mdl_RL.ObjVal - np.max(csell_tmp)*(mdl_RL.getVars()[N].x - mdl_milp.getVars()[N].x)
            # optgap_RL_1.append((np.abs(cost_RL-cost_milp))*100/np.abs(cost_milp))
            
            list_cost_RL1.append(cost_RL)
            list_cost_milp2.append(cost_milp)
        
        if feas_SL == True and feas_RL == True:
            list_SL.append(mdl_SL.Runtime+end_time_SL-start_time_SL)
            list_RL.append(mdl_RL.Runtime+end_time_RL-start_time_RL)
            list_milp.append(mdl_milp.Runtime)
            
            cost_SL = mdl_SL.ObjVal - np.max(csell_tmp)*(mdl_SL.getVars()[N].x - mdl_milp.getVars()[N].x)        
            cost_RL = mdl_RL.ObjVal - np.max(csell_tmp)*(mdl_RL.getVars()[N].x - mdl_milp.getVars()[N].x)
            
            # optgap_SL_2.append((np.abs(cost_SL-cost_milp))*100/np.abs(cost_milp))
            # optgap_RL_2.append((np.abs(cost_RL-cost_milp))*100/np.abs(cost_milp))
            
            list_cost_SL2.append(cost_SL)
            list_cost_RL2.append(cost_RL)
            list_cost_milp3.append(cost_milp)
            
        if feas_SL == False:    cntr_infeas_SL += 1
        if feas_RL == False:    cntr_infeas_RL += 1
        
    avg_time_SL = np.mean(list_SL)
    avg_time_RL = np.mean(list_RL)
    avg_time_milp = np.mean(list_milp)

    relative_percentage_SL = avg_time_SL/avg_time_milp
    relative_percentage_RL = avg_time_RL/avg_time_milp
    infeas_rate_SL = cntr_infeas_SL/N_sim
    infeas_rate_RL = cntr_infeas_RL/N_sim

    print('average time SL: %.4f' % avg_time_SL)
    print('average time RL: %.4f' % avg_time_RL)
    print('average time milp: %.4f' % avg_time_milp)
    print('relative gain SL: %.4f' % relative_percentage_SL)
    print('relative gain RL: %.4f' % relative_percentage_RL)
    print('infeasibility rate SL: %.4f' % infeas_rate_SL)
    print('infeasibility rate RL: %.4f' % infeas_rate_RL)

    # optimality_gap_SL_1 = np.mean(optgap_SL_1)
    # optimality_gap_RL_1 = np.mean(optgap_RL_1)
    # optimality_gap_SL_2 = np.mean(optgap_SL_2)
    # optimality_gap_RL_2 = np.mean(optgap_RL_2)

    # print('optimality_gap_SL: %.2f, %.2f' %(optimality_gap_SL_1, optimality_gap_SL_2))
    # print('optimality_gap_RL: %.2f, %.2f \n\n'%(optimality_gap_RL_1, optimality_gap_RL_2))
    
    opt_gap_SL1 = (np.mean(list_cost_SL1)-np.mean(list_cost_milp1))*100/np.mean(list_cost_milp1)
    opt_gap_RL1 = (np.mean(list_cost_RL1)-np.mean(list_cost_milp2))*100/np.mean(list_cost_milp2)
    opt_gap_SL2 = (np.mean(list_cost_SL2)-np.mean(list_cost_milp3))*100/np.mean(list_cost_milp3)
    opt_gap_RL2 = (np.mean(list_cost_RL2)-np.mean(list_cost_milp3))*100/np.mean(list_cost_milp3)
    
    print('optimality_gap_SL: %.2f, %.2f' %(opt_gap_SL1, opt_gap_SL2))
    print('optimality_gap_RL: %.2f, %.2f \n\n'%(opt_gap_RL1, opt_gap_RL2))
        
    with open('final_comparison_log.txt', 'a') as f:
        f.write('N = %d, N_sim = %d, year = 2022 \n' %(N,N_sim))
        f.write('average time SL: %.4f\n' % avg_time_SL)
        f.write('average time RL: %.4f\n' % avg_time_RL)
        f.write('average time milp: %.4f\n' % avg_time_milp)
        f.write('relative gain SL: %.4f\n' % relative_percentage_SL)
        f.write('relative gain RL: %.4f\n' % relative_percentage_RL)
        f.write('infeasibility rate SL: %.4f\n' % infeas_rate_SL)
        f.write('infeasibility rate RL: %.4f\n' % infeas_rate_RL)
        # f.write('optimality_gap_SL: %.2f, %.2f\n' %(optimality_gap_SL_1, optimality_gap_SL_2))
        # f.write('optimality_gap_RL: %.2f, %.2f \n\n'%(optimality_gap_RL_1, optimality_gap_RL_2))
        f.write('optimality_gap_SL: %.2f, %.2f\n' %(opt_gap_SL1, opt_gap_SL2))
        f.write('optimality_gap_RL: %.2f, %.2f \n\n'%(opt_gap_RL1, opt_gap_RL2))
        f.close()

average time SL: 0.0014
average time RL: 0.0017
average time milp: 0.0044
relative gain SL: 0.3107
relative gain RL: 0.3900
infeasibility rate SL: 0.0317
infeasibility rate RL: 0.0013
optimality_gap_SL: 0.13, 0.13
optimality_gap_RL: 1.77, 1.72 


average time SL: 0.0017
average time RL: 0.0036
average time milp: 0.0063
relative gain SL: 0.2674
relative gain RL: 0.5729
infeasibility rate SL: 0.0760
infeasibility rate RL: 0.0033
optimality_gap_SL: 0.08, 0.08
optimality_gap_RL: 2.00, 1.95 




In [31]:
len(list_cost_RL1), len(list_cost_milp2)

(1000, 1000)

In [38]:
np.mean(list_cost_milp2), np.mean(list_cost_RL2)

(543.8545872563586, 553.4098713834026)

In [35]:
tmp = np.mean(np.array(list_cost_RL1) - np.array(list_cost_milp2))*100/np.mean(list_cost_milp2)
tmp

0.819382056137925

In [23]:
gamma = 0
seq_len = N
batch_size = 32

num_layers = 1
n_actions = 16
update_target = 100
max_mem_size = 20000
DDQN = True
noisyNet = False

match N:
        case 4:
            hidden_size = 64
            state_opt = 5
        case 12:
            hidden_size = 256
            state_opt = 3
        case 24:
            hidden_size = 512
            state_opt = 4
        case 48:
            hidden_size = 512
            state_opt = 4

match state_opt:
    case 1|2:
        input_size=6
    case 3|4:
        input_size=4*N+1
    case 5:
        input_size=5*N+1

h0_RL = torch.zeros(num_layers, 1, hidden_size)
c0_RL = torch.zeros(num_layers, 1, hidden_size)

agent = DQN_Agent(gamma=gamma, lr=lr, input_size=input_size, seq_len=seq_len, batch_size=batch_size,
              hidden_size=hidden_size, num_layers=num_layers, n_actions=n_actions, update_target=update_target, max_mem_size=max_mem_size, DDQN=DDQN, noisyNet=noisyNet)

match N:
    case 4:
        agent.Q_eval.load_state_dict(torch.load('best_weights\job_N04_029_24_2_24_16_47_22'))
    case 12:
        agent.Q_eval.load_state_dict(torch.load('best_weights\job_N12_27'))
    case 24:
        agent.Q_eval.load_state_dict(torch.load('best_weights\job_N24_19_23_12_6_17_44_19'))
    case 48:       
        agent.Q_eval.load_state_dict(torch.load('best_weights\job_N48_103_24_2_6_2_26_14'))            


match N:
    case 4:
        cost_lower_bound=-866
        cost_upper_bound=1838
    case 12:
        cost_lower_bound=-1832
        cost_upper_bound=5452
    case 24:
        cost_lower_bound=-1980
        cost_upper_bound=10694
    case 48:
        cost_lower_bound=-32
        cost_upper_bound=19171

N_sim=3000

avg_cost_lstm, avg_cost_optimal, optimality_gap, cntr_infeasible = get_optimality_gap(agent,Env, N_sim, cbuy, csell, cprod, power_load,
                                                                                      power_res, state_min, state_max, state_opt, cost_lower_bound, cost_upper_bound)

print(avg_cost_lstm, avg_cost_optimal, optimality_gap*100, cntr_infeasible)

554.2723082669598 549.5792945469885 0.8539284078814683 0


In [ ]:
# N_iter =  100

# list_RL = []
# list_milp = []

# # agent.epsilon = 0
# # agent.Q_eval.eval()

# for i in range(N_iter):

#     Env.set_randState()
    
#     start_time_RL = time.time()    
#     state = build_stacked_input_v2(Env.state, Env.idx_cntr, agent.seq_len, state_min2=state_min, state_max2=state_max,
#                                         cbuy=cbuy, csell=csell, cprod=cprod, power_load=power_load, power_res=power_res)
                    
#     action_idx = agent.choose_action(state, greedy=True)

#     delta = Env.build_delta_vector(action_idx)
#     end_time_RL = time.time()

#     # mdl = gurobi_qp(Env.state, N,
#     #                     power_res[Env.idx_cntr:Env.idx_cntr+N],
#     #                     power_load[Env.idx_cntr:Env.idx_cntr+N],
#     #                     cbuy[Env.idx_cntr:Env.idx_cntr+N],
#     #                     csell[Env.idx_cntr:Env.idx_cntr+N],
#     #                     cprod[Env.idx_cntr:Env.idx_cntr+N],
#     #                     delta)
    
#     # info = step(Env, action_idx, cbuy, csell, cprod, power_load, power_res, cost_upper_bound=cost_upper_bound, cost_lower_bound=cost_lower_bound)
#     info = Env.step(action_idx, cbuy, csell, cprod, power_load, power_res, cost_upper_bound=cost_upper_bound, cost_lower_bound=cost_lower_bound)[4]
        
#     # feas = qp_feasible(mdl)
#     # mdl = info['mdl']
#     feas = info['feasible']
        
#     start_time_milp = time.time()
#     mdl_milp = hybrid_fhocp(Env.state, N,
#                         power_res[Env.idx_cntr:Env.idx_cntr+N],
#                         power_load[Env.idx_cntr:Env.idx_cntr+N],
#                         cbuy[Env.idx_cntr:Env.idx_cntr+N],
#                         csell[Env.idx_cntr:Env.idx_cntr+N],
#                         cprod[Env.idx_cntr:Env.idx_cntr+N])
#     feas_milp = qp_feasible(mdl_milp)
#     end_time_milp = time.time()
    
#     if feas==True:
#         # list_RL.append(end_time_RL-start_time_RL)
#         # list_milp.append(end_time_milp-start_time_milp)
#         mdl = info['mdl']
#         list_RL.append(mdl.Runtime+end_time_RL-start_time_RL)
#         list_milp.append(mdl_milp.Runtime)
    
    
# avg_time_RL = np.mean(list_RL)
# avg_time_milp = np.mean(list_milp)

# # relative_percentage = 1-(avg_time_milp - avg_time_RL)/avg_time_milp
# relative_percentage = avg_time_RL/avg_time_milp

# print('average time RL: %.4f' % np.mean(list_RL))
# print('average time milp: %.4f' % np.mean(list_milp))
# print('relative gain: %.4f' % relative_percentage)
